### DeAp Decorating Classes

Tzn. nebudeme mít classu co je dekorátor - ale budeme dekorovat classy! Tj. docela velkej rozdíl.

In [319]:
from fractions import Fraction

In [320]:
f = Fraction(2,3)

In [321]:
f.speak()

AttributeError: 'Fraction' object has no attribute 'speak'

Tak naučme zlomky mluvit!

In [322]:
Fraction.speak = 100

In [323]:
f.speak

100

Ale to je jen hodnota, zkusíme mu dát funkci, callable.

In [324]:
Fraction.speak = lambda self, message: f"Fraction says {message}"

In [325]:
f.speak("This is a late parrot")

'Fraction saysThis is a late parrot'

Trochu pro ujasnění, naše lambda má self - objet, classa a message tzn. parametr, který mu zadáváme. Takže naše frakce je sama o sobě objekt - a naučíme classu takto metodu speak.

Monkey patching = právě dodávání atribut "in runntime"

#### Frakce jestli je inetgral

In [327]:
Fraction.is_integral = lambda self: self.denominator == 1

In [328]:
f1 = Fraction(2,3)
f2 = Fraction(64,8)

Celé číslo je pokud je dělitel 1 - cokoliv co dělím 1 je celé, jasné.

In [330]:
f1.is_integral(), f2.is_integral()

(False, True)

Moc pěkné :) - máme ověření zda je frakce celé číslo. Zkusíme pomocí deco.

In [331]:
def dec_speak(cls): #funkci předáváme classu
    cls.speak = lambda self, message: f"{self.__class__.__name__} says {message}"
    
    return cls


In [332]:
Fraction = dec_speak(Fraction)

In [333]:
f1 = Fraction(2,3)

In [334]:
f1.speak("Hello")

'Fraction says Hello'

In [335]:
class Person:
    pass

In [336]:
Person = dec_speak(Person)

In [337]:
p = Person()

In [338]:
p.speak("this work")

'Person says this work'

V podstatě zde ukazuji nástin jak vše funguje. Definuji že moje funkce speak bere jako parametr classu a tudíž ta classa když je volaná tak zavolá jméno classy a zprávu kterou zadám. Stejný princip jako u dekorátorů.

#### Chceme dodat debugging info pro classu

In [340]:
from datetime import datetime, timezone

In [349]:
def info(self): #self je zde důležité! Jako kdyby to byla classa
    """zde pouze řeším jaké informace přidávám do listu
    neboli jaké bude naše debugging info
    #čas - jméno a id"""
    results = []
    results.append(f"time: {datetime.now(timezone.utc)}")
    results.append(f"Class {self.__class__.__name__}")
    results.append(f"id: {hex(id(self))}")
        
    #vars - chceme atributy classy získat - spešl fičura a items
    #pro získání klíčů a hodnot
    for k,v in vars(self).items():
        results.append(f"{k}: {v}")
    return results

def debug_info(cls):
    cls.debug = info
    return cls

#nakonec jsem rozděllil na dvě části, máme funkci info - doluje prostě jen data
#a debug_info ta bude používat info pro dolování dat z classy

In [350]:
@debug_info
class Person:
    def __init__(self, name, birth_year):
        self.name = name
        self.birth_year = birth_year
        
    def say_hi():
        return "hello there"

In [351]:
p = Person("John", 1939)

In [352]:
p.debug()

['time: 2020-06-03 04:37:48.226791+00:00',
 'Class Person',
 'id: 0x1d0b8021d00',
 'name: John',
 'birth_year: 1939']

Takže mám dekorovanou classu :D. Tzn. moje classa zavolá debug info a ta zavolá info z funkce info.

#### Nyní pokus bez navracení classy

In [353]:
def info2(self): #self je zde důležité! Jako kdyby to byla classa
    """zde pouze řeším jaké informace přidávám do listu
    neboli jaké bude naše debugging info
    #čas - jméno a id"""
    results = []
    results.append(f"time: {datetime.now(timezone.utc)}")
    results.append(f"Class {self.__class__.__name__}")
    results.append(f"id: {hex(id(self))}")
        
    #vars - chceme atributy classy získat - spešl fičura a items
    #pro získání klíčů a hodnot
    for k,v in vars(self).items():
        results.append(f"{k}: {v}")
    return results

def debug_info2(cls):
    cls.debug = info2
#zadávám si 2ky pro sychr

In [354]:
@debug_info
class Person2:
    def __init__(self, name, birth_year):
        self.name = name
        self.birth_year = birth_year
        
    def say_hi():
        return "hello there"

In [355]:
s = Person2("John2", 1940)

In [356]:
s.debug()

['time: 2020-06-03 04:41:25.237897+00:00',
 'Class Person2',
 'id: 0x1d0b82a8d30',
 'name: John2',
 'birth_year: 1940']

Neukazujeme nic jiného jen to že debug_info2 nemusí nic navracet. Náš deco se ale dá použít na více věcí takto.

In [360]:
@debug_info
class Car:
    def __init__(self, make, model, year, top_speed):
        self.make = make
        self.model = model
        self.year = year
        self.top_speed = top_speed
        self._speed =  0
        #pseudo private
    #vytvořím speed na kterou mohu použít set a get
    @property #built-in deco pro metody
    def speed(self):
        return self._speed
    
    @speed.setter
    def speed(self, new_speed):
        if new_speed > self.top_speed:
            raise ValueError("Speed cannon exceed top_speed")
        else:
            self._speed = new_speed
    #settery a gettery jsou trochu mimo prozatím
        

In [358]:
favorite = Car("Ford","Model T", 1908, 45)

In [359]:
favorite.debug()

['time: 2020-06-03 05:32:15.672613+00:00',
 'Class Car',
 'id: 0x1d0b84bff10',
 'make: Ford',
 'model: Model T',
 'year: 1908',
 'top_speed: 45',
 '_speed: 0']

A mám, zde krásné info ohledně všeho co je třeba :).

In [361]:
from math import sqrt

In [367]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __abs__(self):
        """speciální dunder funkce pro absolutní"""
        return sqrt(self.x**2 + self.y**2)
    
    def __repr__(self):
        return f"Point{self.x}, {self.y}"
#v podstatě mám jen classu o x a y ..

In [363]:
p1, p2, p3 = Point(2,3), Point(2,3), Point(0,0)

In [368]:
abs(p1) #metoda funguje

3.605551275463989

In [369]:
p1 is p2 #jasné mají jinou referenci

False

In [370]:
p1 == p2 #nemáme qual metodu nelze porovnávat classy

False

In [375]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __abs__(self):
        """speciální dunder funkce pro absolutní"""
        return sqrt(self.x**2 + self.y**2)
    
    def __repr__(self):
        return f"Point{self.x}, {self.y}"
    
    def __eq__(self, other):
        """dunder pro rovnost"""
        if isinstance(other, Point):
        #pokud se jedná o point - rovnost nastane takto
            return self.x == other.x and self.y == other.y
        else:
            return False
    def __lt__(self, other):
        """dunder pro porovnávání"""
        if isinstance(other, Point):
            return abs(self) < abs(other)
        #tehle zápis mě docela prudí - Trochu se zamyslet proč tohle takto funguje
        #viz na řádku 378 ..Ono mu stačí je < ..protože Python je schopný si to přetočit
        #takže to pozná ..--> ALE <= => NENÍ podporováno
        else:
            return NotImplemented

In [376]:
p1, p2, p3 = Point(2,3), Point(2,3), Point(0,0)

In [377]:
p1 == p2

True

Nyní nám to vše funguje, ale musíme dodat ordering.

In [378]:
p3 < p1

True

In [380]:
p3 > p1 #python si to přepíše jako p1 < p3

False

In [381]:
#ale
p1 => p2

SyntaxError: invalid syntax (<ipython-input-381-b9f3fb8e915d>, line 2)

Nemáme. Museli bychom definovat další dunder - le, gt, ge, ne apod.

* a <= b iff (a<b or a = b)
* a > b iff not (a<b) and a != b
* a >= b iff not (a<b)

V podstatě jen zápis že mi stačí rovnost a < - pro tvorbu ostatních částí

In [394]:
def complete_ordering(cls):
    if '__eq__' in dir(cls) and '__lt__' in dir(cls):
        #pokud dáš dir na clasuu má zápis takto i s ''
        cls.__le__ = lambda self, other: self < other or self == other
        cls.__gt__ = lambda self, other: not (self<other) and not (self == other)
        cls.__ge__ = lambda self, other: not (self<other)
    return cls
#takže používám funkci jako deco kde mám 3 lambdy pro definování rovností - docela mindfuck
#lambda funkce jsou asi jasné dvě hodnoty self a other - a podmínky pro implementace jen 
#pomocí < a ==
# například toto znamená: not (self<other) and not (self == other)
#dunder greater then nastane pokud --> není menší a není rovno -> 
#tak je logicky větší než :)

In [389]:
@complete_ordering
#díky tomuto pokud budu chtít po classe porovnání ve stavu gt, lt, ge 
#tak jelikož už má implementováno < znaménkko z lt :D a == z eq 
#tak pokud jsem schopný zapsat ostatní rovnosti jen na základě těchto znamének
#mohu tak dodělat dekorátor
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __abs__(self):
        """speciální dunder funkce pro absolutní"""
        return sqrt(self.x**2 + self.y**2)
    
    def __repr__(self):
        return f"Point{self.x}, {self.y}"
    
    def __eq__(self, other):
        """dunder pro rovnost"""
        if isinstance(other, Point):
        #pokud se jedná o point - rovnost nastane takto
            return self.x == other.x and self.y == other.y
        else:
            return False
    def __lt__(self, other):
        """dunder pro porovnávání"""
        if isinstance(other, Point):
            return abs(self) < abs(other)
        else:
            return NotImplemented

In [390]:
p1, p2, p3, p4 = Point(2,3), Point(2,3), Point(0,0), Point(100,200)

In [395]:
p1 <= p4
#zavolá dunder metodu __lt__ ale zavolá jí z decorátoru!

True

In [393]:
p1!=p2

False

Tzn. náš dekorátor nám obohatil classu o možnost porovnávání.

Python má fičuru co to zmákne sama.

In [397]:
from functools import total_ordering
#jedná se v pdostatě o decorátor

In [400]:
#
#@total_ordering 
#stačí aby měl pouze jeden a zbytek zvládne sám 
#moc pěkná fičura tedy
#
#Takže to nemusím budovat od začátku
#